Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

Download the data text8 as we did in the previous preoject.

In [2]:
url = 'http://mattmahoney.net/dc/'
##Download the file
def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


Connect all the words into a long string.

In [3]:
#Connect all the words into a large string
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Let's have a look at the string.

In [4]:
text[:100]

' anarchism originated as a term of abuse first used against early working class radicals including t'

Create a small validation set.

In [5]:
#Split the string into training and validating parts
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [6]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

In [7]:
vocabulary_size

27

Create some functions to map words into ID numbers and vice versa.

In [8]:
#Map a char into an ID
def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
#Map an ID into a char  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [9]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    self._segment = self._text_size // batch_size
    #Cursor stores the indice of characters in the original text
    self._cursor = [ offset * self._segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
    
  def _test_cursor(self):
    return self._cursor

  def _test_segment(self):
    return self._segment
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    #batch is an array of batch_size * vocabulary_size(64*27)
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    #one-hot encoder, encode each character as [000100...]
    #Note, first pick the ith character then then (i+segment)th character
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

In [10]:
def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
#print(batches2string(valid_batches.next()))
#print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [11]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In a traditional recurrent neural network, during the gradient back-propagation phase, the gradient signal can end up being multiplied a large number of times (as many as the number of timesteps) by the weight matrix associated with the connections between the neurons of the recurrent hidden layer. This means that, the magnitude of weights in the transition matrix can have a strong impact on the learning process.

If the weights in this matrix are small (or, more formally, if the leading eigenvalue of the weight matrix is smaller than 1.0), it can lead to a situation called vanishing gradients where the gradient signal gets so small that learning either becomes very slow or stops working altogether. It can also make more difficult the task of learning long-term dependencies in the data. Conversely, if the weights in this matrix are large (or, again, more formally, if the leading eigenvalue of the weight matrix is larger than 1.0), it can lead to a situation where the gradient signal is so large that it can cause learning to diverge. This is often referred to as exploding gradients.

These issues are the main motivation behind the LSTM model which introduces a new structure called a memory cell (see Figure 1 below). A memory cell is composed of four main elements: an input gate, a neuron with a self-recurrent connection (a connection to itself), a forget gate and an output gate. The self-recurrent connection has a weight of 1.0 and ensures that, barring any outside interference, the state of a memory cell can remain constant from one timestep to another. The gates serve to modulate the interactions between the memory cell itself and its environment. The input gate can allow incoming signal to alter the state of the memory cell or block it. On the other hand, the output gate can allow the state of the memory cell to have an effect on other neurons or prevent it. Finally, the forget gate can modulate the memory cell’s self-recurrent connection, allowing the cell to remember or forget its previous state, as needed.

Simple LSTM Model, as displayed below:
![image](LSTM/lstm_memorycell.png)

The equations below describe how a layer of memory cells is updated at every timestep t. In these equations :
- x(t) is the input to the memory cell layer at time t
- Wi, Wf, Wc, Wo, Ui, Uf, Uc, Uo and Vo are weight matrices
- bi, bf, bc and bo are bias vectors

First, we compute the values for i(t), the input gate, and C~(t) the candidate value for the states of the memory cells at time t:

![image](LSTM/11.png)

![image](LSTM/22.png)
 
Second, we compute the value for f(t), the activation of the memory cells’ forget gates at time t :


![image](LSTM/33.png)

Given the value of the input gate activation i(t), the forget gate activation f(t) and the candidate state value C~(t), we can compute C(t) the memory cells’ new state at time t :


![image](LSTM/44.png)

With the new state of the memory cells, we can compute the value of their output gates and, subsequently, their outputs :


![image](LSTM/55.png)


![image](LSTM/66.png)


In [80]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters(weights):
  # Input gate: input, previous output, and bias.
  #weights for the input data x(t)
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  #weights for the last output h(t-1)
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  #biase
  ib = tf.Variable(tf.zeros([1, num_nodes]))#bias
  # Forget gate: input, previous output, and bias.
  #weights for the input i(t)
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  #weights for last output h(t-1)
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.    
  #weights for the input data x(t)
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  #weights for the last output h(t-1)
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases, multiply the output of LSTM cell and get the
  # predictionss
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)#i(t)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)#f(t)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    candidate_state = tf.tanh(update) #C~(t)   
    state = forget_gate * state + input_gate * candidate_state#C(t)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)#O(t)
    output = output_gate * tf.tanh(state)#h(t)
    return output, state

  # Input data, a list of batches of characters in the form of one-hot
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  # Assign new value(current outpput) to saved_output
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    #Connect all the outputs and map the output of LSTM to predictions
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  #???
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [12]:
num_steps = 7001
summary_frequency = 500

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.293868 learning rate: 10.000000
Minibatch perplexity: 26.95
uvnimhmhkm  bgfpnordaeaqudv iozgjxwttu  kalhy  ee ynoih  wkrq eavafxepzdskrxekd 
vigtrljnjhynpaq pairs nhcvf ve  vzwnysripelinpepsz qneeodahdeoaa ykpglnjdfyuebdf
yzeo  rejlfrdttosiaayzgdnfocjpkztc pprart  vtlemtkn pzei pvkremzi  bfumlzerp  jo
wzloien qfypxnm prrsaeqdprqwo f  jncslsdb ki tscecuhel rx dxiznv n iadttuu xwebq
x fpunaxnggwxbcrgchnbtograslvatfaaamavdexpknl  cvefhclccd qcau ae dxs elkiosujyn
Validation set perplexity: 20.22
Average loss at step 500: 2.176252 learning rate: 10.000000
Minibatch perplexity: 6.44
Validation set perplexity: 7.26
Average loss at step 1000: 1.848792 learning rate: 10.000000
Minibatch perplexity: 5.61
Validation set perplexity: 6.01
Average loss at step 1500: 1.748836 learning rate: 10.000000
Minibatch perplexity: 4.78
Validation set perplexity: 5.40
Average loss at step 2000: 1.69

In [71]:
predictions.shape

(1, 27)

In [72]:
labels.shape

(640, 27)

In [73]:
np.log(predictions).shape

(1, 27)

In [66]:
logprob(predictions, labels)

4.7389280297793448

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [30]:
def createVariables(row_num, col_num):
    return tf.Variable(tf.truncated_normal([row_num, col_num], -0.1, 0.1))

In [33]:
num_nodes = 64

graph1 = tf.Graph()
with graph1.as_default():
  
  # Parameters(weights):
  # Input gate: input, previous output, and bias.
  #weights for the input data x(t)
  ix = createVariables(vocabulary_size, num_nodes)
  #weights for the last output h(t-1)
  im = createVariables(num_nodes, num_nodes)
  #biase
  ib = tf.Variable(tf.zeros([1, num_nodes]))#bias
  # Forget gate: input, previous output, and bias.
  #weights for the input i(t)
  fx = createVariables(vocabulary_size, num_nodes)
  #weights for last output h(t-1)
  fm = createVariables(num_nodes, num_nodes)
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.    
  #weights for the input data x(t)
  cx = createVariables(vocabulary_size, num_nodes)
  #weights for the last output h(t-1)
  cm = createVariables(num_nodes, num_nodes)
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = createVariables(vocabulary_size, num_nodes)
  om = createVariables(num_nodes, num_nodes)
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases, multiply the output of LSTM cell and get the
  # predictionss
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    i_variable = tf.concat(1, [ix, fx, cx, ox])
    o_variable = tf.concat(1, [im, fm, cm, om])
    input_product, output_product = tf.matmul(i, i_variable), tf.matmul(o, o_variable)
    ix_product = input_product[:,:num_nodes]
    io_product = output_product[:,:num_nodes]
    fx_product = input_product[:,num_nodes:2*num_nodes]
    fo_product = output_product[:,num_nodes:2*num_nodes]
    cx_product = input_product[:,2*num_nodes:3*num_nodes]
    co_product = output_product[:,2*num_nodes:3*num_nodes]
    ox_product = input_product[:,3*num_nodes:]
    oo_product = output_product[:,3*num_nodes:]
    input_gate = tf.sigmoid(ix_product + io_product + ib)#i(t)
    forget_gate = tf.sigmoid(fx_product + fo_product + fb)#f(t)
    update = cx_product + co_product + cb
    candidate_state = tf.tanh(update) #C~(t)   
    state = forget_gate * state + input_gate * candidate_state#C(t)
    output_gate = tf.sigmoid(ox_product + oo_product + ob)#O(t)
    output = output_gate * tf.tanh(state)#h(t)
    return output, state

  # Input data, a list of batches of characters in the form of one-hot
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  # Assign new value(current outpput) to saved_output
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    #Connect all the outputs and map the output of LSTM to predictions
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  #???
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [44]:
num_steps = 701
summary_frequency = 100

with tf.Session(graph=graph1) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.294983 learning rate: 10.000000
Minibatch perplexity: 26.98
o elmufaeprsrni  yxetnafn dgiciqkbveokn mkvohaasafoib gr f nzeasnsij t cvevzrppk
dpribkshkp bngsolay vddbeqemgkttflbiedcsqw opzdezs xg siqatop cfmhpaesjtbggnjttb
aislneynnjea n icso tnnnamfoupcfliwhvn ohaeo  d edemrarwteobmo  sh sijackbldeapz
 te taraowzljmwf wqlebdsjgr yw   smomzfoewucenbt e eloos pa s  t hwkehejhr e wox
afnozqnxovv eqja iszvxduak  ypupbnrt zavsdatb ot hot o rh enla  otndvgsxrapmbira
Validation set perplexity: 20.05
Average loss at step 100: 2.576816 learning rate: 10.000000
Minibatch perplexity: 10.55
Validation set perplexity: 11.04
Average loss at step 200: 2.237416 learning rate: 10.000000
Minibatch perplexity: 8.85
Validation set perplexity: 8.98
Average loss at step 300: 2.100085 learning rate: 10.000000
Minibatch perplexity: 8.07
Validation set perplexity: 8.06
Average loss at step 400: 2.019947 learning rate: 10.000000
Minibatch perplexity: 7.97
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

## Create bigram training pairs

First, we create bigram pairs for those letter including the blank ' '.

In [12]:
batch_size=128
num_unrollings=10
pair_size = vocabulary_size * vocabulary_size
##creat ID for each letter pairs, for example, 'aa':0#
def createPairDic(letter1, letter2):
    if letter1 is None or letter2 is None:
        print('Empty letters')
        return None
    pairDict = dict()
    i = 0
    for l in letter1:
        for m in letter2:
            pairDict[(l,m)] = i
            i += 1
    reverse_pairDict = dict(zip(pairDict.values(), pairDict.keys())) 
    return pairDict, reverse_pairDict

In [13]:
letters = string.ascii_lowercase+' '
pairDict, reverse_pairDict = createPairDic(letters, letters)

Now, we create a function to map letters into ID.

In [14]:
#Map the character pairs into Ids
def pair2id(pairDict, letter1, letter2):
    if letter1 not in letters or letter2 not in letters:
        print('Unexpected Character')
        return None
    return pairDict[(letter1, letter2)]
def id2pair(reverse_pairDict, id):
    if id<0 or id>len(reverse_pairDict)-1:
        print('Wrong ID for Character Pair')
        return None
    l, m = reverse_pairDict[id]
    return l+m

In [15]:
id2pair(reverse_pairDict, 4)

'ae'

In [16]:
train_text[:64]

'ons anarchists advocate social relations based upon voluntary as'

Let's generate batches of character pairs such as 'ac', 'bv', 'cd', and the labels are like 'a', 'c', 'g'.

In [17]:
class PairBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    self._segment = self._text_size // batch_size
    #Cursor stores the indice of characters in the original text
    self._cursor = [ offset * self._segment for offset in range(batch_size)]
    self._last_batch, self._last_label = self._next_train_label()
    
  def _test_cursor(self):
    return self._cursor

  def _test_segment(self):
    return self._segment

  def _next_train_label(self):
      batch = np.zeros(shape=(self._batch_size), dtype=np.int)
      label = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.int)
      #Note, first pick the ith character then then (i+segment)th character
      for b in range(self._batch_size):
            #Input: a pair such as 'ab'
            letter1 = self._text[self._cursor[b]]
            letter2 = self._text[(self._cursor[b]+1)%self._text_size]
            batch[b] = pair2id(pairDict, letter1, letter2)
            #Output: a letter such as 'c'
            label[b, char2id(self._text[(self._cursor[b]+2)%self._text_size])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
      return batch, label
 
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    labels = [self._last_label]
    for step in range(self._num_unrollings-1):
        batch, label = self._next_train_label()
        batches.append(batch)
        labels.append(label)
    self._last_batch = batches[-1]
    self._last_label = labels[-1]
    return batches, labels

In [18]:
def vec2pair(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2pair(reverse_pairDict, c) for c in np.argmax(probabilities, 1)]

def pairs(batch):
  """Turn a code back into a pair of characters representation."""
  return [id2pair(reverse_pairDict, c) for c in batch]

def pairs2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  #s=''
  for b in batches:
    s = [''.join(x) for x in zip(s, pairs(b))]
    #s += id2pair(reverse_pairDict,b)+','
  return s

In [19]:
train_batches = PairBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = PairBatchGenerator(valid_text, 1, 1)

In [20]:
batches, labels = train_batches.next()

In [21]:
print(batches2string(labels))

['s anarchis', 'mination g', 'en militar', 'hree nine ', 'eria arche', 'viated as ', 'bbeys and ', 'ing the ri', 'rried urra', 'ty upset t', 'l and rich', 'ed in the ', 'and liturg', 'isgust bec', ' opened fo', 'ciety and ', 'on from th', 'o based ch', 'gration to', 'ero zero f', 'w york oth', 'ort subjec', ' boeing se', 'ow two you', 'listed wit', ' during th', 'er has pro', 'ot dead na', 'be made to', ' s enthusi', 'r who rece', 'erates thr', 'e signific', 'ines secur', 'fierce cri', 'uel extrac', 'wo six eig', 'ure that w', 'istotle s ', 'dragas con', 'y can be l', 'ombinant r', 'nd intrace', 'nsive manu', 'on of the ', ' attack fr', ' to pass h', ' to bring ', 'certain dr', 'ench janse', ' it will t', 'on from eu', 'convince t', 'her sponta', 't told him', 'gest partn', 'paign and ', ' in a spec', 'er side st', 'in the amp', 'us texts s', 'ssignment ', 'capitalize', 'ttas franc', 'duplicate ', 'rmer is wi', ' ann es d ', 'the series', 'e january ', ' it is pos', 'ss zero th', 'worl

In [22]:
print(pairs2string(batches))

['onnss  aannaarrcchhi', 'noommiinnaattiioonn ', 'whheenn  mmiilliitta', ' tthhrreeee  nniinne', 'llleerriiaa  aarrcch', 'reevviiaatteedd  aas', ' aabbbbeeyyss  aannd', 'shhiinngg  tthhee  r', 'maarrrriieedd  uurrr', 'siittyy  uuppsseett ', 'heell  aanndd  rriic', 'asseedd  iinn  tthhe', 'y  aanndd  lliittuur', ' ddiissgguusstt  bbe', 'ayy  ooppeenneedd  f', 'soocciieettyy  aannd', 'tiioonn  ffrroomm  t', 'aggoo  bbaasseedd  c', 'miiggrraattiioonn  t', ' zzeerroo  zzeerroo ', 'neeww  yyoorrkk  oot', 'shhoorrtt  ssuubbjje', 'hee  bbooeeiinngg  s', 'sggooww  ttwwoo  yyo', 'e  lliisstteedd  wwi', 'ltt  dduurriinngg  t', 'ebbeerr  hhaass  ppr', ' nnoott  ddeeaadd  n', 'o  bbee  mmaaddee  t', 'lll  ss  eenntthhuus', 'yeerr  wwhhoo  rreec', 'oppeerraatteess  tth', 'orree  ssiiggnniiffi', 'rmmiinneess  sseeccu', 'a  ffiieerrccee  ccr', ' ffuueell  eexxttrra', ' ttwwoo  ssiixx  eei', 'attuurree  tthhaatt ', 'arriissttoottllee  s', 'e  ddrraaggaass  cco', 'ittyy  ccaann  bbee ', 'eccoommbbiinna

In [23]:
def pair_sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def pair_sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, pair_size], dtype=np.float)
  p[0, pair_sample_distribution(prediction[0])] = 1.0
  return p

def pair_random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, pair_size])
  return b/np.sum(b, 1)[:,None]

In [25]:
num_nodes = 64
embedding_size = 30 # Dimension of the embedding vector.
graph2 = tf.Graph()
with graph2.as_default():
  
  # Parameters(weights):
  # Input gate: input, previous output, and bias.
  #weights for the input data x(t)
  ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  #weights for the last output h(t-1)
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  #biase
  ib = tf.Variable(tf.zeros([1, num_nodes]))#bias
  # Forget gate: input, previous output, and bias.
  #weights for the input i(t)
  fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  #weights for last output h(t-1)
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.    
  #weights for the input data x(t)
  cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  #weights for the last output h(t-1)
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases, multiply the output of LSTM cell and get the
  # predictionss
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)#i(t)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)#f(t)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    candidate_state = tf.tanh(update) #C~(t)   
    state = forget_gate * state + input_gate * candidate_state#C(t)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)#O(t)
    output = output_gate * tf.tanh(state)#h(t)
    return output, state

  # Input data, a list of batches of characters in the form of one-hot
  embeddings = tf.Variable(
    tf.random_uniform([pair_size, embedding_size], -0.1, 0.1))  
  train_inputs = list()
  train_labels = list()
  for _ in range(num_unrollings):
    #Input character pairs
    train_inputs.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
    #Character Labels
    train_labels.append(tf.placeholder(tf.int32, shape=[batch_size, vocabulary_size]))

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state  
  for i in train_inputs:
    embed = tf.nn.embedding_lookup(embeddings, i)
    output, state = lstm_cell(embed, output, state)
    outputs.append(output)

  # State saving across unrollings.
  # Assign new value(current outpput) to saved_output
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    #Connect all the outputs and map the output of LSTM to predictions
    #Calculate the logits
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    #Calculate the loss
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))


  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    1.0, global_step, 1000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  #???
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)#The size: numnode*rollnum*embedsize
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])#Each time input a character pair
  embed = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [26]:
characters(sample(random_distribution()))

['w']

In [27]:
num_steps = 8001
summary_frequency = 200
with tf.Session(graph=graph2) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches, labels = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings):
      feed_dict[train_inputs[i]] = batches[i]
      feed_dict[train_labels[i]] = labels[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))

      mean_loss = 0
      labels = np.concatenate(labels)
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
    
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          #Generate a character pair randomly
          feed = np.random.choice(range(pair_size))#Select an id randomly
          sentence = id2pair(reverse_pairDict, feed)#Map it into a pair
          reset_sample_state.run()
          for _ in range(79):
            feed = np.array([feed])
            prediction = sample_prediction.eval({sample_input: feed})
            char = characters(sample(prediction))[0]#Map the prediction into a character
            feed = pair2id(pairDict, sentence[-1], char)#Next pair
            #feed = sample(prediction)
            sentence += char
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      #reset_sample_state.run()
      #valid_logprob = 0
      #for _ in range(valid_size):
       # b = valid_batches.next()
        #predictions = sample_prediction.eval({sample_input: b[0]})
        #valid_logprob = valid_logprob + logprob(predictions, b[1])
     # print('Validation set perplexity: %.2f' % float(np.exp(
        #valid_logprob / valid_size)))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.295720 learning rate: 1.000000
Minibatch perplexity: 27.00
jnqswxowfwaiwvpmoamqlzadreqgmaorzjcdoxf ljrcmeccnioofjdmerqiwglglfcymihnnawhkihkl
gkadszhgz qjtjgxojmqjqxffntmincoiioewf noetgswelvdnccxnkzur bwoaohoajwbssjkxq dwv
 rksvgqxoxmzriwjmjellek wxso uuaxcajnowqg jzpkqttdfpaji orxxgbkfhuntbnca rmyzkikd
arcqxzyyuqucuthtnskiqjultaxvruvqajjjy  bvcfmscskbulhzhuhedybqqpixyanjujsepnmooqaf
ktowgo nphcsnxzfjfkqjzj xvtknnntkwojfflplnbpuyz tgthqjxaiwvpja cmhiwjceyewmilxcof
Average loss at step 200: 2.840556 learning rate: 1.000000
Minibatch perplexity: 14.82
Average loss at step 400: 2.566505 learning rate: 1.000000
Minibatch perplexity: 11.43
Average loss at step 600: 2.341649 learning rate: 1.000000
Minibatch perplexity: 9.78
Average loss at step 800: 2.243043 learning rate: 1.000000
Minibatch perplexity: 8.61
Average loss at step 1000: 2.184154 learning rate: 0.100000
Minibatch per

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---